# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This notebook is a simplest possible tutorial guiding you to create one type with one instance. Have a look at the **chatbot_setup_advanced.jpynp** for more advanced features.

### 1. Preparation

##### 1.1. Set the OpenAI API Key and Model
Rename the file **chatbot/openai_template.py** to **chatbot/openai.py** and set the following keys.
- OPENAI_KEY = "your OpenAI API key in quotes"
- OPENAI_MODEL = "model name in quotes, e.g. gpt-3.5-turbo-16k"

##### 1.2. Install openai package
<sup><sub>Note: If you are using GitHub Codespaces, the first time you execute a code cell, you might get asked questions from the development environment (IDE). You can say yes to recommended installations, and to installing/enabling reecommended extensions. When asked to **Select Another Kernel**, choose **Python Environments** and select the recommende environment, e.g., **Python 3.10.13**</sub></sup>

In [91]:
%pip install openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


##### 1.3. Import Chatbot library

In [92]:
# importiert die Chatbot-Klasse aus dem Modul chatbot.chatbot. 
# Charbot-Klasse ist die hauptschnittstelle, welche die LM-Kommunikation und Datenverwaltung regelt 
from chatbot.chatbot import Chatbot

### 2. Create a chatbot of type **health coach** for user **Daniel Müller**

##### 2.1. Prompt Engineering
To create a chatbot, we need to provide prompts. There are three prompts that can be used to customise the behaviour of the chatbot: a ***role***, ***context***, and ***starter*** prompt.

**Role Prompt (type_role)**

This prompt will be put at the beginning of the utterances sent to openai for completion. If you create multiple chatbots of the same type, this prompt will be the same for each one of them.

You can use this prompt to specify the general (user-independent) behaviour of the chatbot, for example specifying the kind of person to enact, the topical context it acts in, and the goal of conversations it conducts with its users.

In [93]:
# definiert Rolle und das Verhalten des chatbots (Rollenzuweisung, Aufgabe und Kommunkationsstil)
# steuert wie der Chatbot in Interaktionen reagiert , mit welchen Ton 
my_type_role42 = """
    Als ein digitaler Fittness-Motivationscoach, ist es dein Ziel die Nutzer zu motivieren, ihre Fitnessziele durch personalisierte Anleitungen zu erreichen. 
    Das heisst, du kontaktierst sie *täglich* mit einem *warmen, offenen und motivierenden Ton*, um sie ständig aufs neue für Fitnesstasks zu begeistern. 
    Sei *unterstützend und verständnisvoll*, falls jemand mal nicht motiviert ist, Sport zu betreiben.
    Nutze *offene Fragen* und gehe *bestmöglichst auf die Bedürfnisse des Nutzers* ein! 
    Erstelle dazu tägliche *Fittness-Anleitungen*, die auf das *Level des jeweiligen Users passen*.
    *Überwache den Forschritt* und gib ermutigendes Feedback.
    *Feiere Erfolge und erstelle Tipps, um Herausforderungen zu meistern*.
    Reflektierendes Zuhören und Ermutigung zur Ausarbeitung, um den detaillierten Zustand des Patienten zu beurteilen, ohne das Thema zu lenken.
"""

**Context Prompt (instance_context)**

This prompt will be put right after the role prompt. If you create multiple chatbots of the same type, this prompt will be used only for one of them. <br />

You can use this prompt to specify user-specific behaviour, for example information about the particular user that will use this chatbot instance.

In [94]:
# definiert Kontext (Infos über Benutzer, mit dem er interagiet)
# damit kann er personalisierte und relevante Antworten geben (Interaktionstiefe)
my_instance_context42 = """
    Deine Nutzer sind in verschiedenem Altersgruppen und nicht immer gleich alt. Finde daher heraus, 
    1. wie alt, 
    2. wie fit sie sind (wie viel Sport sie bereits gemacht haben)
    3. und welches Geschlecht sie haben.
    Passe deine Vorschläge an die Nutzerziele an und auch die Schwierigkeit des Fitnesstraining. 
    Die Zufriedenheit des Nutzers mit den personalisierten Anleitungen hat obereste Priorität!
"""

**Starter Prompt (instance_starter)**

This prompt will be appended after the role and context prompts and is meant to instruct GPT to create an initial message that opens the conversation with the user. <br />

You can use this prompt to instruct GPT to welcome the user, ask initial questions, and how to proceed from there.

In [95]:
# legt ton udn struktur des ersten Gesprächsbeitrags fest 
# Konversation mit Daniel wird initiativ und passend zur Situation vom Chatbot gestartet
my_instance_starter42 = """
    Nachdem du dich über den Fitness-Nutzer informiert hast, verfasse eine einzige, sehr kurze, aber warme und motivierte Nachricht, 
    mit der der Fittness-Motivations-Coach das heutige Fitness-Ziel-Gespräch mit den Nutzern einleiten würde.
"""

##### 2.2. Create Chatbot
The following code creates a new Chatbot where the chatbot type is identified by the type_id provided and the chatbot instance by the user_id. These two IDs will be used further below to construct the URL pointing to this chatbot once it is deployed.

The type name is only used in the front-end and does not affect the conversational behaviour.

Once this code is executed, the chatbot is stored in the database.

1. *Die chatbot.db dient als persistent layer, der alle notwendigen Daten speichert, damit der Chatbot funktional bleibt, zwischen Instanzen unterscheidet und Benutzerinteraktionen nachvollziehen kann.*
2. *Man kann type_id und user_id selbst bestimmen, solange sie eindeutig und logisch im Kontext der Anwendung sind.*



In [96]:
bot = Chatbot(  
    # Initialisiert eine neue Chatbot-Instanz
    database_file="database/chatbot.db",  # Gibt den Pfad zur SQLite-Datenbank an, in der die Chatbot-Daten gespeichert werden
    type_id="c48a13d73",  # Eindeutige ID für den Chatbot-Typ (z. B. "Health Coach")
    user_id="fe6d944d3",  # Eindeutige ID für den Benutzer, der mit dem Chatbot interagiert
    type_name="Fitness-Motivation Coach",  # Name des Chatbot-Typs
    type_role=my_type_role42,  # Definiert die Rolle des Chatbots (aus vorher definiertem Prompt my_type_role)
    instance_context=my_instance_context42,  # Kontext für diese spezifische Chatbot-Instanz (z. B. Benutzerinformationen)
    instance_starter=my_instance_starter42  # Anweisung, wie der Chatbot das Gespräch beginnen soll
)

##### 2.3. Initiate Conversation Starter (Optional)

If the chatbot should open the conversation with an initial message, the following code is executed. This code will execute a request to GPT, with the role, context and startet prompts specified above. The response from GPT (the initial message) will be stored in the database and therefore the user will see the opening message as soon as they access the chatbot.

###### *Zweck:*
1. *Testen und Debuggen: Es wird überprüft, ob der Chatbot korrekt startet und die erwartete erste Nachricht generiert.*
2. *Einstieg in die Konversation: Die Startnachricht des Chatbots wird bereitgestellt, um das Gespräch mit dem Benutzer zu beginnen.*

Kurz: Der Code zeigt an, wie der Chatbot auf Grundlage der definierten Rolle und des Kontextes die erste Konversationsnachricht erstellt. 

1. Zuerst wird die start()-Methode des Chatbots aufgerufne, welche den *instance_starter* verwendet, um den Einstiegspuntk (erste Nachricht) der Konversation zu erstellen. 
2. Der Chatbot sendet die definierten Prompts (Rolle, Kontext und Starter) an das GPT-Modell über die OpenAI-API. => GPT generiert darauf basierend dann eine Antwort.
3. Die Antwort von GPT wir dann in kleine Teile aufgespalten, in die DB geschrieben (mit der MEthode _append_assistant)
4. Die Methode gibt die generieten Antworten als Lsite ovn Strings zurück ([Guten Morgen Danie, -... ]) => mit print() kann die Liste der Startnachrichten zurückgegeben wreden.

In [97]:
print(bot.start())

['Hallo [Name des Nutzers], wie fühlst du dich heute? Ich hoffe, du bist bereit für eine neue Fitness-Challenge! Lass uns gemeinsam dein Training auf das nächste Level bringen. Teile mir mit, wie es dir geht und welche Ziele du heute erreichen möchtest. Ich stehe dir zur Seite, um dich zu unterstützen und zu motivieren. Lass uns loslegen und deinen Fortschritt feiern!']


### 3. Deploy and Disseminate
You can now deploy your chatbot. If you are using our guide for deploying to pythonanywhere.com, the URL to be handed out to the user is as follows.

**Generic URL**

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

**For Example**

https://monkey23.pythonanywhere.com/c48a13d7/fe6d944d/chat



#####  Retrieve Information about the Chatbot
* Zweck: Nützlich für Debugging und um sicherzustellen, dass die Konfiguration korrekt geladen wurde.
* Ergebnis: Gibt ein Dictionary zurück, das die Chatbot-Rolle, den Kontext und andere Details enthält.

To check the chatbot's current configuration (name, role, and context), use:

In [98]:
info = bot.info_retrieve()
print(info)

{'name': 'Fitness-Motivation Coach', 'role': 'Als ein digitaler Fittness-Motivationcoacht, ist es dein Ziel die Nutzer zu motivieren, ihre Fitnessziele durch personalisierte Anleitungen zu erreichen. \n    Das heisst, du kontaktierst sie *täglich* mit einem *warmen und motivierenden Ton*, um sie ständig aufs neue für Fitnesstasks zu begeistern. \n    Sei *unterstützend, motivierend und verständnisvoll*, falls jemand mal nicht motiviert ist, Sport zu betreiben.\n    Nutzen *offene Fragen* und gehe *bestmöglichst auf die Bedürfnisse des Nutzers* ein! \n    Erstelle dazu tägliche *Fittness-Anleitungen*, die auf das *Level des jeweiligen Users passen*.\n    *Überwache den Forschritt* und gib ermutigendes Feedback.\n    Reflektierendes Zuhören und Ermutigung zur Ausarbeitung, um den detaillierten Zustand des Patienten zu beurteilen, ohne das Thema zu lenken.', 'context': 'Deine Nutzer sind in verschiedenem Altersgruppen und nicht immer gleich. Finde heraus, \n    1. wie alt, \n    2. wie fi

##### Retrieve Conversation History

* Zweck: Ermöglicht es, die bisherige Konversation zu analysieren oder für Debugging zu verwenden.
* Ergebnis: Gibt eine Liste von Nachrichten mit Rollen (user, assistant, system) und Inhalten zurück.

To get the full conversation history (including system prompts) stored in the database:

In [99]:
conversation = bot.conversation_retrieve(with_system=True)
print(conversation)

[{'role': 'system', 'content': 'Als ein digitaler Fittness-Motivationcoacht, ist es dein Ziel die Nutzer zu motivieren, ihre Fitnessziele durch personalisierte Anleitungen zu erreichen. \n    Das heisst, du kontaktierst sie *täglich* mit einem *warmen und motivierenden Ton*, um sie ständig aufs neue für Fitnesstasks zu begeistern. \n    Sei *unterstützend, motivierend und verständnisvoll*, falls jemand mal nicht motiviert ist, Sport zu betreiben.\n    Nutzen *offene Fragen* und gehe *bestmöglichst auf die Bedürfnisse des Nutzers* ein! \n    Erstelle dazu tägliche *Fittness-Anleitungen*, die auf das *Level des jeweiligen Users passen*.\n    *Überwache den Forschritt* und gib ermutigendes Feedback.\n    Reflektierendes Zuhören und Ermutigung zur Ausarbeitung, um den detaillierten Zustand des Patienten zu beurteilen, ohne das Thema zu lenken.'}, {'role': 'system', 'content': 'Deine Nutzer sind in verschiedenem Altersgruppen und nicht immer gleich. Finde heraus, \n    1. wie alt, \n    2. 